In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

In [ ]:
train = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train" 
val ="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val"
test = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Test"

gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2,)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (220, 220))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(220, 220))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(220, 220))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights = 'imagenet', input_shape = (220, 220,3), include_top = False)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                           

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2,restore_best_weights=True)

history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 11, callbacks = callback)

Epoch 1/11
40/40 [==============================] - 921s 23s/step - loss: 0.3783 - accuracy: 0.8270 - val_loss: 0.2842 - val_accuracy: 0.8546
Epoch 2/11
40/40 [==============================] - 351s 9s/step - loss: 0.2962 - accuracy: 0.8727 - val_loss: 0.3282 - val_accuracy: 0.8393
Epoch 3/11
40/40 [==============================] - 216s 5s/step - loss: 0.2323 - accuracy: 0.8988 - val_loss: 0.2246 - val_accuracy: 0.8947
Epoch 4/11
40/40 [==============================] - 136s 3s/step - loss: 0.2320 - accuracy: 0.8965 - val_loss: 0.2439 - val_accuracy: 0.8947
Epoch 5/11
40/40 [==============================] - 100s 2s/step - loss: 0.2384 - accuracy: 0.8977 - val_loss: 0.2324 - val_accuracy: 0.8892


In [ ]:
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 1, callbacks = callback)

40/40 [==============================] - 62s 2s/step - loss: 0.2702 - accuracy: 0.8820 - val_loss: 0.2823 - val_accuracy: 0.8463


In [ ]:
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 1, callbacks = callback)

40/40 [==============================] - 57s 1s/step - loss: 0.2278 - accuracy: 0.9086 - val_loss: 0.2803 - val_accuracy: 0.8629


In [ ]:
model.save_weights("MobileNet_v2_3.h5")

In [ ]:
y_pred = model.predict(test_gen)

12/12 [==============================] - 297s 27s/step


In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])



<ipython-input-11-ccac160ef34b>:4: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))


In [ ]:
y_actual = []

for i in range(len(y_pred)):
  if i <361 :
    y_actual.append(0)
  else:
    y_actual.append(1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,y_pred_list)

0.8199445983379502

In [ ]:
np.save("Mobilenet-history.npy",history.history)
model.save("Mobilenet.h5")

In [ ]:
from google.colab import files 
files.download("Mobilenet.h5")

files.download("Mobilenet-history.npy")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>